In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox, Frame, Label, Entry, Button, StringVar, simpledialog
import pyodbc
import requests
from datetime import datetime
from PIL import Image, ImageTk  
import io

class BibliotecaApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Sistema de Biblioteca - Professional Edition")
        self.root.geometry("1000x700")
        self.root.configure(bg="#f0f2f5")
        
        # Lista completa de gêneros literários
        self.generos_literarios = [
            "Arte", "Autoajuda", "Aventura", "Biografia", "Ciência", 
            "Ciência Política", "Contos", "Crítica Literária", "Culinária", 
            "Didático", "Direito", "Distopia", "Economia", "Ensaios", 
            "Espiritualidade", "Esportes", "Fantasia", "Ficção", 
            "Ficção Científica", "Filosofia", "Geografia", "História", 
            "História em Quadrinhos", "Horror", "Humor", "Infantil", 
            "Jornalismo", "Juvenil", "LGBTQ+", "Linguística", 
            "Literatura Africana", "Literatura Asiática", "Literatura Brasileira", 
            "Literatura Estrangeira", "Literatura Feminista", "Matemática", 
            "Memórias", "Mistério", "Mitologia", "Música", "Não-Ficção", 
            "Novela", "Policial", "Poesia", "Psicologia", "Religião", 
            "Romance", "Saúde", "Sociologia", "Suspense", "Terror", 
            "Thriller", "Turismo", "Viagem", "Young Adult"
        ]
        
        self.setup_db()
        self.create_widgets()  # Primeiro cria todos os widgets principais
        self.criar_barra_busca()  # Depois cria a barra de busca
        self.carregar_dados()
        
    def setup_db(self):
        try:
            self.conn = pyodbc.connect(
                'DRIVER={ODBC Driver 17 for SQL Server};'
                'SERVER=localhost\\SQLEXPRESS;'
                'DATABASE=BibliotecaLivros;'
                'Trusted_Connection=yes;'
            )
            self.cursor = self.conn.cursor()
            self.create_tables()
        except Exception as e:
            messagebox.showerror("Erro de Banco de Dados", f"Não foi possível conectar ao banco:\n{str(e)}")
            self.root.destroy()

    def create_tables(self):
        try:
            self.cursor.execute("""
                IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='Livros')
                CREATE TABLE Livros (
                    ID INT IDENTITY(1,1) PRIMARY KEY,
                    Titulo NVARCHAR(255) NOT NULL,
                    Autor NVARCHAR(255) NOT NULL,
                    AnoPublicacao INT,
                    Genero NVARCHAR(100),
                    Disponibilidade BIT DEFAULT 1,
                    DataCadastro DATETIME DEFAULT GETDATE()
                )
            """)
            self.conn.commit()
        except Exception as e:
            messagebox.showwarning("Aviso", f"Erro ao verificar tabelas:\n{str(e)}")

    def create_widgets(self):
        self.setup_styles()
        self.create_header()
        self.create_form_frame()
        self.create_table_frame()
        self.create_button_frame()  # Cria o button_frame antes de criar_barra_busca()
        self.create_status_bar()

    def setup_styles(self):
        style = ttk.Style()
        style.configure("TFrame", background="#f0f2f5")
        style.configure("TLabel", background="#f0f2f5", font=('Arial', 10))
        style.configure("TButton", font=('Arial', 10, 'bold'), padding=6)
        style.map("TButton",
                foreground=[('active', 'white'), ('!disabled', 'white')],
                background=[('active', '#0056b3'), ('!disabled', '#007bff')])
        style.configure("Treeview", font=('Arial', 10), rowheight=28)
        style.configure("Treeview.Heading", font=('Arial', 11, 'bold'))

    def create_header(self):
        header = Frame(self.root, bg="#343a40", height=60)
        header.pack(fill="x")
        
        Label(header, 
             text="Sistema de Gestão Bibliotecária", 
             font=('Arial', 16, 'bold'), 
             fg="white", 
             bg="#343a40").pack(side="left", padx=20)
        
        self.lbl_usuario = Label(header, 
                               text="Usuário: Admin", 
                               font=('Arial', 10), 
                               fg="white", 
                               bg="#343a40")
        self.lbl_usuario.pack(side="right", padx=20)

    def create_form_frame(self):
        form_frame = Frame(self.root, bg="#f0f2f5", padx=15, pady=15)
        form_frame.pack(fill="x")
        
        # Título
        Label(form_frame, text="Título*:", bg="#f0f2f5").grid(row=0, column=0, sticky="e", padx=5, pady=5)
        self.titulo_entry = Entry(form_frame, font=('Arial', 10), width=40)
        self.titulo_entry.grid(row=0, column=1, padx=5, pady=5, sticky="w")
        
        # Autor
        Label(form_frame, text="Autor*:", bg="#f0f2f5").grid(row=0, column=2, sticky="e", padx=5, pady=5)
        self.autor_entry = Entry(form_frame, font=('Arial', 10), width=30)
        self.autor_entry.grid(row=0, column=3, padx=5, pady=5, sticky="w")
        
        # Ano e Gênero
        Label(form_frame, text="Ano:", bg="#f0f2f5").grid(row=1, column=0, sticky="e", padx=5, pady=5)
        self.ano_entry = Entry(form_frame, font=('Arial', 10), width=10)
        self.ano_entry.grid(row=1, column=1, sticky="w", padx=5, pady=5)
        
        Label(form_frame, text="Gênero:", bg="#f0f2f5").grid(row=1, column=2, sticky="e", padx=5, pady=5)
        self.genero_combobox = ttk.Combobox(form_frame, 
                                          values=self.generos_literarios,
                                          font=('Arial', 10),
                                          width=25)
        self.genero_combobox.grid(row=1, column=3, sticky="w", padx=5, pady=5)

    def create_table_frame(self):
        table_frame = Frame(self.root, bg="#f0f2f5", padx=15)
        table_frame.pack(fill="both", expand=True)
        
        scroll_y = ttk.Scrollbar(table_frame)
        scroll_y.pack(side="right", fill="y")
        
        scroll_x = ttk.Scrollbar(table_frame, orient="horizontal")
        scroll_x.pack(side="bottom", fill="x")
        
        self.tabela = ttk.Treeview(
            table_frame,
            columns=("ID", "Título", "Autor", "Ano", "Gênero", "Disponível"),
            show="headings",
            yscrollcommand=scroll_y.set,
            xscrollcommand=scroll_x.set,
            selectmode="extended"
        )
        
        colunas = [
            ("ID", 50),
            ("Título", 250),
            ("Autor", 150),
            ("Ano", 60),
            ("Gênero", 150),
            ("Disponível", 80)
        ]
        
        for col, width in colunas:
            self.tabela.heading(col, text=col)
            self.tabela.column(col, width=width, anchor="center")
        
        self.tabela.pack(fill="both", expand=True)
        
        scroll_y.config(command=self.tabela.yview)
        scroll_x.config(command=self.tabela.xview)
        
        self.tabela.bind("<Double-1>", self.editar_livro)

    def create_button_frame(self):
        self.button_frame = Frame(self.root, bg="#f0f2f5", padx=15, pady=10)
        self.button_frame.pack(fill="x")
        
        btn_style = {'font': ('Arial', 10, 'bold'), 'padx': 12, 'pady': 8}
        
        self.btn_adicionar = Button(self.button_frame, 
                                  text="Adicionar Livro", 
                                  command=self.adicionar_livro,
                                  bg="#28a745",
                                  fg="white",
                                  **btn_style)
        self.btn_adicionar.pack(side="left", padx=5)
        
        self.btn_limpar = Button(self.button_frame, 
                               text="Limpar Formulário", 
                               command=self.limpar_formulario,
                               bg="#6c757d",
                               fg="white",
                               **btn_style)
        self.btn_limpar.pack(side="left", padx=5)
        
        self.btn_buscar_api = Button(self.button_frame, 
                                   text="Buscar na API", 
                                   command=self.buscar_por_titulo_api,
                                   bg="#17a2b8",
                                   fg="white",
                                   **btn_style)
        self.btn_buscar_api.pack(side="left", padx=5)
        
        self.btn_remover = Button(self.button_frame, 
                                text="Remover Selecionado", 
                                command=self.remover_livro,
                                bg="#dc3545",
                                fg="white",
                                **btn_style)
        self.btn_remover.pack(side="right", padx=5)
        
        self.btn_atualizar = Button(self.button_frame, 
                                  text="Atualizar Lista", 
                                  command=self.carregar_dados,
                                  bg="#ffc107",
                                  fg="#212529",
                                  **btn_style)
        self.btn_atualizar.pack(side="right", padx=5)

    def criar_barra_busca(self):
        frame_busca = Frame(self.root, bg="#f0f2f5", padx=5, pady=5)
        frame_busca.pack(fill="x", before=self.button_frame)
        
        Label(frame_busca, text="Buscar:", bg="#f0f2f5").pack(side="left", padx=5)
        
        self.entry_busca = Entry(frame_busca, font=('Arial', 10), width=40)
        self.entry_busca.pack(side="left", padx=5)
        
        btn_buscar = Button(frame_busca, 
                          text="Filtrar", 
                          command=lambda: self.carregar_dados(self.entry_busca.get()),
                          bg="#17a2b8",
                          fg="white",
                          font=('Arial', 9, 'bold'))
        btn_buscar.pack(side="left", padx=5)
        
        btn_limpar = Button(frame_busca, 
                          text="Limpar Filtro", 
                          command=lambda: [self.entry_busca.delete(0, 'end'), self.carregar_dados()],
                          bg="#6c757d",
                          fg="white",
                          font=('Arial', 9, 'bold'))
        btn_limpar.pack(side="left", padx=5)

    def create_status_bar(self):
        self.status_var = StringVar()
        self.status_var.set("Pronto")
        
        status_bar = Frame(self.root, bg="#e9ecef", height=30)
        status_bar.pack(fill="x", side="bottom")
        
        Label(status_bar, 
             textvariable=self.status_var, 
             font=('Arial', 9), 
             bg="#e9ecef").pack(side="left", padx=10)
        
        Label(status_bar, 
             text=f"Sistema Biblioteca v1.0 | {datetime.now().year}", 
             font=('Arial', 9), 
             bg="#e9ecef").pack(side="right", padx=10)

    def carregar_dados(self, filtro=None):
        try:
            for item in self.tabela.get_children():
                self.tabela.delete(item)
            
            sql = "SELECT ID, Titulo, Autor, AnoPublicacao, Genero, Disponibilidade FROM Livros"
            params = ()
            
            if filtro:
                sql += " WHERE Titulo LIKE ? OR Autor LIKE ?"
                params = (f'%{filtro}%', f'%{filtro}%')
            
            sql += " ORDER BY Titulo"
            
            self.cursor.execute(sql, params)
            livros = self.cursor.fetchall()
            
            for livro in livros:
                id, titulo, autor, ano, genero, disp = livro
                status = "Sim" if disp else "Não"
                self.tabela.insert("", "end", values=(id, titulo, autor, ano, genero, status))
            
            self.status_var.set(f"Carregados {len(livros)} livros" + (f" (Filtro: '{filtro}')" if filtro else ""))
            
        except Exception as e:
            messagebox.showerror("Erro", f"Falha ao carregar dados:\n{str(e)}")
            self.status_var.set("Erro ao carregar dados")

    def limpar_formulario(self):
        self.titulo_entry.delete(0, tk.END)
        self.autor_entry.delete(0, tk.END)
        self.ano_entry.delete(0, tk.END)
        self.genero_combobox.set('')
        self.status_var.set("Formulário limpo")

    def validar_campos(self):
        if not self.titulo_entry.get().strip():
            messagebox.showwarning("Validação", "O campo Título é obrigatório!")
            self.titulo_entry.focus()
            return False
            
        if not self.autor_entry.get().strip():
            messagebox.showwarning("Validação", "O campo Autor é obrigatório!")
            self.autor_entry.focus()
            return False
            
        return True

    def adicionar_livro(self):
        if not self.validar_campos():
            return
            
        try:
            titulo = self.titulo_entry.get().strip()
            autor = self.autor_entry.get().strip()
            ano = int(self.ano_entry.get()) if self.ano_entry.get().strip() else None
            genero = self.genero_combobox.get().strip()
            
            self.cursor.execute("""
                INSERT INTO Livros (Titulo, Autor, AnoPublicacao, Genero)
                VALUES (?, ?, ?, ?)
            """, (titulo, autor, ano, genero or None))
            
            self.conn.commit()
            self.carregar_dados()
            self.limpar_formulario()
            self.status_var.set(f"Livro '{titulo}' adicionado com sucesso!")
            
        except ValueError:
            messagebox.showerror("Erro", "O ano deve ser um número válido!")
            self.ano_entry.focus()
        except Exception as e:
            messagebox.showerror("Erro", f"Falha ao adicionar livro:\n{str(e)}")

    def editar_livro(self, event):
        item = self.tabela.selection()
        if not item:
            return
            
        id_livro = self.tabela.item(item, 'values')[0]
        
        try:
            self.cursor.execute("SELECT * FROM Livros WHERE ID = ?", (id_livro,))
            livro = self.cursor.fetchone()
            
            if not livro:
                messagebox.showerror("Erro", "Livro não encontrado!")
                return
                
            edit_window = tk.Toplevel(self.root)
            edit_window.title("Editar Livro")
            edit_window.grab_set()
            
            Label(edit_window, text="Título*:").grid(row=0, column=0, padx=5, pady=5, sticky="e")
            titulo_edit = Entry(edit_window, width=40)
            titulo_edit.grid(row=0, column=1, padx=5, pady=5)
            titulo_edit.insert(0, livro.Titulo)
            
            Label(edit_window, text="Autor*:").grid(row=1, column=0, padx=5, pady=5, sticky="e")
            autor_edit = Entry(edit_window, width=40)
            autor_edit.grid(row=1, column=1, padx=5, pady=5)
            autor_edit.insert(0, livro.Autor)
            
            Label(edit_window, text="Ano:").grid(row=2, column=0, padx=5, pady=5, sticky="e")
            ano_edit = Entry(edit_window, width=10)
            ano_edit.grid(row=2, column=1, padx=5, pady=5, sticky="w")
            ano_edit.insert(0, livro.AnoPublicacao if livro.AnoPublicacao else "")
            
            Label(edit_window, text="Gênero:").grid(row=3, column=0, padx=5, pady=5, sticky="e")
            genero_edit = ttk.Combobox(edit_window, 
                                     values=self.generos_literarios,
                                     width=25)
            genero_edit.grid(row=3, column=1, padx=5, pady=5, sticky="w")
            genero_edit.set(livro.Genero if livro.Genero else "")
            
            Label(edit_window, text="Disponível:").grid(row=4, column=0, padx=5, pady=5, sticky="e")
            disp_var = tk.BooleanVar(value=livro.Disponibilidade)
            disp_check = tk.Checkbutton(edit_window, variable=disp_var)
            disp_check.grid(row=4, column=1, padx=5, pady=5, sticky="w")
            
            def salvar_edicao():
                if not titulo_edit.get().strip() or not autor_edit.get().strip():
                    messagebox.showwarning("Validação", "Título e Autor são obrigatórios!")
                    return
                    
                try:
                    ano = int(ano_edit.get()) if ano_edit.get().strip() else None
                    
                    self.cursor.execute("""
                        UPDATE Livros 
                        SET Titulo = ?, Autor = ?, AnoPublicacao = ?, Genero = ?, Disponibilidade = ?
                        WHERE ID = ?
                    """, (
                        titulo_edit.get().strip(),
                        autor_edit.get().strip(),
                        ano,
                        genero_edit.get().strip() or None,
                        disp_var.get(),
                        id_livro
                    ))
                    
                    self.conn.commit()
                    edit_window.destroy()
                    self.carregar_dados()
                    self.status_var.set(f"Livro ID {id_livro} atualizado com sucesso!")
                    
                except ValueError:
                    messagebox.showerror("Erro", "Ano deve ser um número válido!")
                except Exception as e:
                    messagebox.showerror("Erro", f"Falha ao atualizar:\n{str(e)}")
            
            Button(edit_window, 
                 text="Salvar", 
                 command=salvar_edicao,
                 bg="#28a745",
                 fg="white").grid(row=5, column=0, columnspan=2, pady=10)
            
        except Exception as e:
            messagebox.showerror("Erro", f"Falha ao carregar dados do livro:\n{str(e)}")

    def remover_livro(self):
        item = self.tabela.selection()
        if not item:
            messagebox.showwarning("Aviso", "Selecione um livro para remover!")
            return
            
        id_livro = self.tabela.item(item, 'values')[0]
        titulo = self.tabela.item(item, 'values')[1]
        
        if not messagebox.askyesno("Confirmar", f"Remover o livro '{titulo}'?"):
            return
            
        try:
            self.cursor.execute("DELETE FROM Livros WHERE ID = ?", (id_livro,))
            self.conn.commit()
            self.carregar_dados()
            self.status_var.set(f"Livro '{titulo}' removido com sucesso!")
        except Exception as e:
            messagebox.showerror("Erro", f"Falha ao remover livro:\n{str(e)}")

    def buscar_por_titulo_api(self):
        titulo = simpledialog.askstring("Buscar Livro", "Digite o título do livro:")
        if not titulo:
            return

        self.status_var.set(f"Buscando '{titulo}' na API...")
        self.root.update_idletasks()

        try:
            url = f"https://openlibrary.org/search.json?title={titulo.replace(' ', '+')}"
            resposta = requests.get(url, timeout=15)
            dados = resposta.json()

            if dados.get('numFound', 0) == 0:
                messagebox.showinfo("Aviso", "Nenhum livro encontrado com este título.")
                self.status_var.set("Busca concluída - nenhum resultado")
                return

            livro = next((item for item in dados['docs'] 
                         if item.get('title') and item.get('author_name')), None)
            
            if not livro:
                messagebox.showinfo("Aviso", "Nenhum livro com informações completas encontrado.")
                self.status_var.set("Busca concluída - dados incompletos")
                return

            self.limpar_formulario()
            self.titulo_entry.insert(0, livro.get('title', ''))
            
            autores = livro.get('author_name', [])
            if autores:
                self.autor_entry.insert(0, ", ".join(autores) if isinstance(autores, list) else autores)
            
            if livro.get('first_publish_year'):
                self.ano_entry.insert(0, str(livro['first_publish_year']))
            
            generos = livro.get('subject', [])
            if generos:
                genero = generos[0] if isinstance(generos, list) else generos
                self.genero_combobox.set(genero)

            self.status_var.set(f"Dados carregados para: {livro.get('title', '')}")
            
            if livro.get('cover_i'):
                capa_url = f"https://covers.openlibrary.org/b/id/{livro['cover_i']}-M.jpg"
                self.mostrar_capa_livro(capa_url)

        except requests.exceptions.RequestException as e:
            messagebox.showerror("Erro de Conexão", f"Falha ao acessar a API:\n{str(e)}")
            self.status_var.set("Erro na conexão com a API")
        except Exception as e:
            messagebox.showerror("Erro", f"Falha ao processar dados:\n{str(e)}")
            self.status_var.set("Erro na busca")

    def mostrar_capa_livro(self, url):
        try:
            resposta = requests.get(url, stream=True, timeout=10)
            if resposta.status_code == 200:
                img = Image.open(io.BytesIO(resposta.content))
                img.thumbnail((150, 200))
                foto = ImageTk.PhotoImage(img)
                
                janela_capa = tk.Toplevel(self.root)
                janela_capa.title("Capa do Livro")
                label_capa = tk.Label(janela_capa, image=foto)
                label_capa.image = foto
                label_capa.pack()
                
        except Exception as e:
            print(f"Erro ao carregar capa: {str(e)}")

if __name__ == "__main__":
    root = tk.Tk()
    app = BibliotecaApp(root)
    root.mainloop()